## MongoDB資料庫連線

In [1]:
import re
from pymongo import MongoClient 

#預設就是自己
client = MongoClient('10.120.28.12',27017)
database = client['test']
collection =database['test2']

#for post in collection.find():
    #print post['title']

## 實戰:給關鍵字進行搜尋+分群

In [2]:
tag1 = "柯文哲"
tag2 = "雙城"

#查詢滿足多重條件的語法是{"$and":[{條件1},{條件2},{條件3},...]}
#另有$or, $not, $nor
#$all在pymongo已廢止, 因為與$and功能重複

for post in collection.find(
    {"$and":[                   
            {"content":{"$regex":tag1}},
            {"content":{"$regex":tag2}},
            {"date":{"$regex":"2016"}}
            ]},{"_id":0}):
    print post['title']

騎鐵馬環台北,柯P挺8選將記仇MG149,痛批費鴻泰「道德有問題」
沿路po臉書,1小時2萬人按讚
「洪慈庸是柯文哲現象擴大」柯連趕4攤輔選「國民黨太低估我」
柯P,莫自比哥倫布
玉山救人是假的?被救者「若無柯P已做頭七」
不及格的,選前行銷
太強了,太痛苦了,28小時騎完雙塔,柯：絕不再幹
柯文哲戰駒,今午1元起標雙城雙塔2車值37萬,標金捐作公益
戰駒飆74萬,柯允陪騎2分鐘
柯戲稱：一日雙塔列員工考核
運動腕錶,健康戴著走價格兩極化發展,挑選看自身需求
義賣戰駒車衣,柯捐335萬車衣面交教練,2單車買家傳是捷安特


In [3]:
#搜尋+計算字詞頻率
import re
import jieba
from sklearn.feature_extraction.text import CountVectorizer
jieba.set_dictionary('E:/dics/dict.txt.big.txt')  #切換至中文繁體字庫
jieba.load_userdict("E:/dics/dict_keyw.txt")       #加入自建詞庫

#給關鍵字----------------------------------------
tag1 = "柯文哲"
tag2 = "雙塔"
#-----------------------------------------------

date=[]
title=[]
content =[]
#把資料庫東西抓出來
for post in collection.find(
    {"$and":[                   
            {"content":{"$regex":tag1}},
            {"content":{"$regex":tag2}},
            #{"date":{"$regex":"2016"}},
            ]},{"_id":0}): 
    summary = post['content']
    content.append('/'.join(jieba.cut(summary)))
    title.append(post['title'])
    date.append(post['date'])
print type(title)
print title[0]
print title[1]
print "---------------------------------"

# 準備計算字詞頻率
# get_feature_names() 可以看"特徵詞有哪些" (『一個字』及『標點符號』會自動被去除掉)
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(content) #裡面要放list
#for i in vectorizer.get_feature_names():
    #print i, #印出特徵詞
print "----------------------"    

#特徵值總共有多少個
print "Feature_num",len(vectorizer.get_feature_names())

#文章數
print "文章數:",len(title)

#透過toarray()可看到詞頻矩陣的结果
print X.toarray()

Building prefix dict from E:\dics\dict.txt.big.txt ...
Loading model from cache c:\users\bigdat~1\appdata\local\temp\jieba.u56ec63d90e89b3354142c30b3eb89a90.cache
Loading model cost 0.487 seconds.
Prefix dict has been built succesfully.


<type 'list'>
柯文哲,周六挑戰北高1日單車行
特選輕盈碳纖車款,值15萬行前集訓1個月,每次練完都好喘
---------------------------------
----------------------
Feature_num 2664
文章數: 25
[[0 2 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 1 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]


In [4]:
#計算TF-IDF()
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer()
tfidf = transformer.fit_transform(X) # 塞入TF-IDF 函數進行計算 #对应的tfidf矩阵

#把權重矩陣存起來
weight = tfidf.toarray()

#整合第某篇新聞的"特徵值"，"count"，"weight"
#a=0
#for i in vectorizer.get_feature_names():
    #if int(X.toarray()[0][a]) > int(0):  #出現次數 > 0 再印出來
        #print i ," ",X.toarray()[0][a]," Weight: ",weight[0][a]
    #a=a+1

In [5]:
# 抓出TOP TF-IDF 
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

#vectorizer = TfidfVectorizer()
#X = vectorizer.fit_transform(lectures)
indices = np.argsort(X.toarray()[0])[::-1]  # transformer = TfidfTransformer()  #X.toarray()[5] 是第幾篇新聞的意思
features = vectorizer.get_feature_names()     # vectorizer = CountVectorizer()
top_n = 30
top_features = [features[i] for i in indices[:top_n]]


a=0
for i in top_features:
    print i,weight[0][indices[a]],
    a=a+1
    
#單獨看前幾名的權重值    
print indices
print weight[0][indices[0]]
print weight[0][indices[1]]
print weight[0][indices[2]]
print weight[0][indices[3]]
print weight[0][indices[4]]
print weight[0][indices[5]]
print weight[0][indices[6]]

高雄 0.295998567626 台北 0.145191661308 柯文哲 0.0679314917054 雲林 0.214628478078 打氣 0.160971358559 挑戰 0.0571950377487 公里 0.0862635102391 台61 0.18162924578 北市 0.14631435432 周六 0.160971358559 騎單車 0.0824878114174 路線 0.121086163853 楠梓 0.121086163853 規劃 0.0899636657272 清晨 0.0899636657272 百八十 0.0899636657272 候選人 0.121086163853 預計 0.0899636657272 凌晨 0.0837709780657 城市 0.107314239039 這次 0.0837709780657 單車挑戰 0.107314239039 有網友 0.0975429028798 小時 0.0575090068261 媒體 0.0899636657272 燈塔 0.0526484799397 幕僚 0.0739996419064 沿途 0.0975429028798 新竹 0.0837709780657 立委候選人 0.121086163853 [2655  758 1556 ..., 1735 1734    0]
0.295998567626
0.145191661308
0.0679314917054
0.214628478078
0.160971358559
0.0571950377487
0.0862635102391


In [6]:
#clustering-1
from sklearn import cluster
c = cluster.KMeans(n_clusters=8)
k_data = c.fit_predict(weight)  # weight 直接拿來用 或放 X 有多算一層TF_IDF 分群效果會好一點

In [34]:
#clustering-2
for idx, grp in enumerate(k_data):
    if grp == 6:
        print date[idx],"  ",title[idx]

20130311    富士康旗下,康法科技爆裁員
20130314    鴻海首發140億日圓公司債
20130322    日媒：郭台銘月初赴日,談iTV訂單
20130601    全球布局加速,鴻海進軍印尼傳與手機通路商簽備忘錄,日顯示觸控研發中心昨成立
20130625    郭董：今年景氣沒有樂觀理由
20130627    郭董：1天拼16小時,鴻海股價隨年齡成長為股價破底道歉,今年每股純益估逾8元
20140618    堺工廠做出成績,郭一吐怨氣
20150323    鴻海擬投資印度Snapdeal取代阿里巴巴,洽談參與10億美元募資案
20150327    夏普獲利拉警報,擬減薪2%~5%
20150624    夏普喊話想續前緣,不會拒絕鴻海援助
20150626    鴻海：今年成長不低於10趴郭台銘自豪：營業額年增4千億,世上有多少公司做得到
20150717    《郭董的進擊》紀錄片,1日生活全都露
20160205    「鴻海是台商」讓案子大加分
20160206    日人曖昧難捉摸,郭董屢踢鐵板
20160206    意志如鋼鐵,郭董被譽「現代成吉思汗」
20160217    買下台灣寬頻,郭董出資2成呂芳銘個人出8成,市場聯想鴻海進軍有線電視
20160219    鴻夏戀再等2周,郭董：不能多說
20160220    鴻夏戀,狂亂的大和魂（郭永興）
20160220    鴻夏戀,狂亂的大和魂（郭永興）
20160226    夏普未先說負債,恐有法律爭議
20160226    鴻夏戀還沒定,國銀宣示要搶貸
20160227    外資看鴻夏戀,短空長多鴻海1~2年獲利被拉低,先動手降評砍目標價
20160303    【壹週刊】鴻夏苦戀　3千億賭局主客易位　郭台銘強勢查帳夏普
20160307    夏普專書搶手,10天賣光3萬本
20160310    懷特解盲股價飆,未申請暫停交易挨罰
20160311    鴻海2月營收,同期次高法人：下半年i7報到,營運將逐季成長
20160330    面板霸主地位奠定,鴻家軍歡慶可期
20160331    郭董後天親自簽約,全程說中文
20160331    鴻夏戀,台廠轉型試金石力抗紅潮唯一方法,《金融時報》：自效率導向轉為創新
20130227    鴻夏戀無

In [9]:
print type(k_data)
print k_data

<type 'numpy.ndarray'>
[0 5 0 2 6 6 6 6 2 6 6 7 1 1 6 2 3 3 6 2 4 4 3 5 3]


## 單純把資料抓出來進行TF-IDF計算

In [47]:
#計算字詞頻率
import re
import jieba
from sklearn.feature_extraction.text import CountVectorizer
jieba.set_dictionary('E:/dics/dict.txt.big.txt')  #切換至中文繁體字庫
jieba.load_userdict("E:/dics/dict_keyw.txt")       #加入自建詞庫

title=[]
content =[]
#把資料庫東西抓出來
for post in collection.find():
    summary = post['title']
    title.append('/'.join(jieba.cut(summary)))
print type(title)
print title[0]
print title[1]
print title[2]
print "---------------------------------"

# 準備計算字詞頻率
# get_feature_names() 可以看"特徵詞有哪些" (『一個字』及『標點符號』會自動被去除掉)
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(title) #裡面要放list
for i in vectorizer.get_feature_names():
    print i, #印出特徵詞
print "----------------------"    

#透過toarray()可看到詞頻矩陣的结果
print X.toarray()

Building prefix dict from E:\dics\dict.txt.big.txt ...
Loading model from cache c:\users\bigdat~1\appdata\local\temp\jieba.u56ec63d90e89b3354142c30b3eb89a90.cache
Loading model cost 0.584 seconds.
Prefix dict has been built succesfully.


KeyboardInterrupt: 

In [75]:
#計算TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer
import numpy as np
transformer = TfidfTransformer()
X = vectorizer.fit_transform(title)  # X 是算好的字詞頻率矩陣
tfidf = transformer.fit_transform(X) # 塞入TF-IDF 函數進行計算
                                     #塞入TF-IDF 函數進行計算(這步算是多的，因為要使用上面算好的 x，利用TfidfVectorize 可省略這步 

#可以看算出來TF-IDF 權重矩陣
print tfidf.toarray()
#把權重矩陣存起來
weight = tfidf.toarray()
print type(weight)
print "---------------------------------------------"
#可以透過以下方法進行取值
print weight[0][1]
print "---------------------------------------------"

#特徵值總共有多少個
print "特徵值:",len(vectorizer.get_feature_names())


#整合"特徵值"，"count"，"weight"
a=0    
import time    
#整合第某篇新聞的"特徵值"，"count"，"weight"
for i in vectorizer.get_feature_names():
    if int(X.toarray()[0][a]) > int(0):  #出現次數 > 0 再印出來
        print "特徵值：",i ," 出現次數: ",X.toarray()[0][a]," 權重值: ",weight[0][a]
    a=a+1
        #time.sleep(0.5)
        

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
<type 'numpy.ndarray'>
---------------------------------------------
0.0
---------------------------------------------
特徵值: 580


## Clustering

In [243]:
from sklearn import cluster
c = cluster.KMeans(n_clusters=15)
k_data = c.fit_predict(weight)  # weight 直接拿來用 或放 X 有多算一層TF_IDF 分群效果會好一點

In [259]:
for idx, grp in enumerate(k_data):
    if grp == 6:
        print title[idx]

燦爛/跨年/,/歡動/全台
《/蘋果/》/民調/：/過去/1/年/，/最/讓/你/感到/不滿/的/是/？
揮別/痛楚/,/我/相信/我能/飛
回歸/職場/,/華隆/人/不/放棄
拒當/英雄/,/「/只求/順利/畢業/」
喪子傷/慟/,/「/期待/司法/正義/」
勇敢/媽/：/前面/有/無限/希望/「/早睡/躲/飢餓/」/,/如願/快樂/出遊
甩/32/公斤/,/「/要/hold/住/身材/」
登山/失溫/,/68/歲/男/生死未卜
迎接/2013/,/玉山/連/3/波/瑞雪/又/一個/冷氣團/,/後天/報到
國道/北上/,/估/下午/湧車潮
帶/超量/菸酒/入境/,/今起/除罪
反/媒體/壟斷/,/上千/學子/守夜/忍受/低溫/「/再/冷/也/要/表達/心聲/」
司馬/觀點/：/中國/孤獨/崛起/,/（/江春/男/）
辦美/免簽/,/上/錯/網站/枉花/千元/代辦/網/一堆/也/叫/ESTA/,/「/./gov/」/才/是/官網
蘋論/：/ROC/是/啥
蘋論/：/政府/怕/啥
露營車/當/禮車/,/蜜月/趴/趴/走/「/不怕/訂/不到/房/」/,/要/在/墾丁/迎/曙光
《/蘋果/數/》/,/「/中國/為/世界/強國/」/,/8/成/大陸/人/不/認同
北斗/導航/,/提供/亞太區/服務
我/很/不爽
商圈/明星臉/,/小/李準/基/最/像/鳳眼/好帥/,/飲料/至少/多賣/3/成
共軍/對台/導彈/失準/,/怪罪/美/GPS1996/台海/危機/揭祕/,/中國/改/自/製/北斗/衛星
空/拋/棉被/,/搶/買
全球/首創/芝心餅/皮/PIZZA/，/好/口碑/全/台/熱銷/大破/千萬/份/！
捷運/廁性/侵/,/狼辯/她/沒/反抗/同學/臉書/重逢/,/騙/出遊/,/害/懷孕
「/想鬧/一下/」/煙火/嚇友/誤燒/6/車
騎樓/嘿咻遭/逮/,/女/誣性/侵/呻吟/未/反抗/,/見警/還落/跑/,/被/揭/賣淫
抽/水毀/鄰宅/,/瓏/山林/判賠/186/萬
低溫/9/℃/,/凍死/摔車/西施
除塵/紙/,/買/1/送/1
20121231/開彩/結果/與/獎金/分配/表
男/走上/國道/,/遭/3/車/輾斃/拖/百米
士官/酒駕/撞/8/車/,/軍方/要警/封口
母悲求/國家/幫管/賊/兒/國一生/累犯/,/嗆/「/偷東西/沒什麼/」
動物/姓氏/,/100/元

## Practice_WordCount & TF-IDF
## 重要觀念:
- TF_IDF是把字詞頻率『標準化』的概念，一篇文章中，某字詞雖然只出現一次，但是卻可能很具代表性，TF-IDF會讓他分數變很高
- vectorizer.get_feature_names()會把『一個字』及『標點符號』自動去除掉

## 套建概念:
- scikit-learn包进行TF-IDF分词权重计算主要用到了两个类：CountVectorizer和TfidfTransformer。其中
- CountVectorizer是通过fit_transform函数将文本中的词语转换为词频矩阵，矩阵元素a[i][j] 表示j词在第i个文本下的词频。即各个词语出现的次数，通过get_feature_names()可看到所有文本的关键字，通过toarray()可看到词频矩阵的结果。简例如下：

In [14]:
#計算字詞頻率
from sklearn.feature_extraction.text import CountVectorizer
import jieba
jieba.set_dictionary('E:/dics/dict.txt.big.txt')  #切換至中文繁體字庫
jieba.load_userdict("E:/dics/dict_keyw.txt")       #加入自建詞庫
corpus = ['上櫃IC設計類股新兵報到柯文哲',
          '陳彥廷上櫃成民國95年，，，john上櫃IC設計類股新兵報到',
          '另外,九齊陳彥廷上櫃成民國95年上櫃IC設計類股新兵報到',
]

y=[]
# 把元素抓出來進行切詞
for i in corpus:
    y.append(' '.join(jieba.cut(i)))

# y list裡面存放切好的字詞['上櫃 ic 設計 類股 新兵 報告','陳彥廷 上櫃 成 上櫃','']
print type(y)
print y[0]
print y[1]
print y[2]
print "---------------------------------"

# 準備計算字詞頻率
# get_feature_names() 可以看"特徵詞有哪些" (『一個字』及『標點符號』會自動被去除掉)
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(y) #裡面要放list
for i in vectorizer.get_feature_names():
    print i, #印出特徵詞
print "----------------------"    

#透過toarray()可看到詞頻矩陣的结果
print X.toarray()


Building prefix dict from E:\dics\dict.txt.big.txt ...
Loading model from cache c:\users\bigdat~1\appdata\local\temp\jieba.u56ec63d90e89b3354142c30b3eb89a90.cache
Loading model cost 0.602 seconds.
Prefix dict has been built succesfully.


<type 'list'>
上櫃 IC設計 類股 新兵 報到 柯文哲
陳彥廷 上櫃 成 民國 95 年 ， ， ， john 上櫃 IC設計 類股 新兵 報到
另外 , 九齊 陳彥廷 上櫃 成 民國 95 年 上櫃 IC設計 類股 新兵 報到
---------------------------------
95 ic設計 john 上櫃 九齊 另外 報到 新兵 柯文哲 民國 陳彥廷 類股 ----------------------
[[0 1 0 1 0 0 1 1 1 0 0 1]
 [1 1 1 2 0 0 1 1 0 1 1 1]
 [1 1 0 2 1 1 1 1 0 1 1 1]]


In [13]:
#計算TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer()
X = vectorizer.fit_transform(y)  # X 是算好的字詞頻率矩陣
tfidf = transformer.fit_transform(X) # 塞入TF-IDF 函數進行計算(這步算是多的，因為要使用上面算好的 x，利用TfidfVectorize 可省略這步 )

#可以看算出來TF-IDF 權重矩陣
print tfidf.toarray()
#把權重矩陣存起來
weight = tfidf.toarray()
print "---------------------------------------------"
#可以透過以下方法進行取值
print weight[0][1]
print "---------------------------------------------"

#整合"特徵值"，"count"，"weight"
#a=0
#for i in vectorizer.get_feature_names():
    #print i ,"  ",X.toarray()[0][a],X.toarray()[1][a],X.toarray()[2][a],"  ",weight[0][a],"  ",weight[1][a],"  ",weight[2][a]
    #a=a+1

[[ 0.          0.35653519  0.          0.35653519  0.          0.
   0.35653519  0.35653519  0.60366655  0.          0.          0.35653519]
 [ 0.32353083  0.25125055  0.42540416  0.5025011   0.          0.
   0.25125055  0.25125055  0.          0.32353083  0.32353083  0.25125055]
 [ 0.29771209  0.2312      0.          0.4624      0.39145563  0.39145563
   0.2312      0.2312      0.          0.29771209  0.29771209  0.2312    ]]
---------------------------------------------
0.356535187468
---------------------------------------------


In [16]:
#值接用TfidfVectorizer算，不經過 CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import jieba
jieba.set_dictionary('E:/dics/dict.txt.big.txt')  #切換至中文繁體字庫
jieba.load_userdict("E:/dics/dict_keyw.txt")       #加入自建詞庫
corpus = ['上櫃IC設計類股新兵報到柯文哲',
          '陳彥廷上櫃成民國95年，，，john上櫃IC設計類股新兵報到',
          '另外,九齊陳彥廷上櫃成民國95年上櫃IC設計類股新兵報到',
]

y=[]
# 把元素抓出來進行切詞
for i in corpus:
    y.append(' '.join(jieba.cut(i)))

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(y)  # titile 放文本
weight = X.toarray()

#特徵值總共有多少個
print "Feature_num",len(vectorizer.get_feature_names())

#透過toarray()可看到詞頻矩陣的结果
print X.toarray()

Building prefix dict from E:\dics\dict.txt.big.txt ...
Loading model from cache c:\users\bigdat~1\appdata\local\temp\jieba.u56ec63d90e89b3354142c30b3eb89a90.cache
Loading model cost 0.613 seconds.
Prefix dict has been built succesfully.


Feature_num 12
[[ 0.          0.35653519  0.          0.35653519  0.          0.
   0.35653519  0.35653519  0.60366655  0.          0.          0.35653519]
 [ 0.32353083  0.25125055  0.42540416  0.5025011   0.          0.
   0.25125055  0.25125055  0.          0.32353083  0.32353083  0.25125055]
 [ 0.29771209  0.2312      0.          0.4624      0.39145563  0.39145563
   0.2312      0.2312      0.          0.29771209  0.29771209  0.2312    ]]


## 抓出 TOP 的TF-IDF 特徵詞 

In [82]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import jieba
jieba.set_dictionary('E:/dics/dict.txt.big.txt')  #切換至中文繁體字庫
jieba.load_userdict("E:/dics/dict_keyw.txt")       #加入自建詞庫
corpus = ['上櫃IC設計類股新兵報到柯文哲',
          '陳彥廷上櫃成民國95年，，，john上櫃IC設計類股新兵報到',
          '另外,九齊陳彥廷上櫃成民國95年上櫃IC設計類股新兵報到',
]

y=[]
# 把元素抓出來進行切詞
for i in corpus:
    y.append(' '.join(jieba.cut(i)))

# y list裡面存放切好的字詞['上櫃 ic 設計 類股 新兵 報告','陳彥廷 上櫃 成 上櫃','']
print type(y)
print y[0]
print y[1]
print y[2]
print "---------------------------------"

#印出所有關鍵詞
for i in vectorizer.get_feature_names():
    print i


#vectorizer = TfidfVectorizer()
#X = vectorizer.fit_transform(y)
indices = np.argsort(transformer.idf_)[::-1]
features = vectorizer.get_feature_names()
top_n = 2
top_features = [features[i] for i in indices[:top_n]]
for i in top_features:
    print i

Building prefix dict from E:\dics\dict.txt.big.txt ...
Loading model from cache c:\users\bigdat~1\appdata\local\temp\jieba.u56ec63d90e89b3354142c30b3eb89a90.cache
Loading model cost 0.547 seconds.
Prefix dict has been built succesfully.


<type 'list'>
上櫃 IC設計 類股 新兵 報到 柯文哲
陳彥廷 上櫃 成 民國 95 年 ， ， ， john 上櫃 IC設計 類股 新兵 報到
另外 , 九齊 陳彥廷 上櫃 成 民國 95 年 上櫃 IC設計 類股 新兵 報到
---------------------------------
95
ic設計
john
上櫃
九齊
另外
報到
新兵
柯文哲
民國
陳彥廷
類股


IndexError: list index out of range

## 補充：抓取TOP 的 TF-IDF

In [87]:
#全部的特徵值一起算
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(y)  # titile 放文本
indices = np.argsort(vectorizer.idf_)[::-1] #[::-1] hello倒著寫 → olleh
features = vectorizer.get_feature_names()
top_n = 2
top_features = [features[i] for i in indices[:top_n]]    # indices[:top_n]] 這邊改成單獨的 list sort 就可以抓出單篇的 TF-IDF 

for i in top_features:
    print i
    
    
print type(indices)  #
print indices        # 這個東西根據特徵詞的Weight進行大小的排序，例如 8 排第一名，所以代表第 8 個的特徵詞weight表現最好
print X.toarray()

柯文哲
另外
<type 'numpy.ndarray'>
[ 8  5  4  2 10  9  0 11  7  6  3  1]
[[ 0.          0.35653519  0.          0.35653519  0.          0.
   0.35653519  0.35653519  0.60366655  0.          0.          0.35653519]
 [ 0.32353083  0.25125055  0.42540416  0.5025011   0.          0.
   0.25125055  0.25125055  0.          0.32353083  0.32353083  0.25125055]
 [ 0.29771209  0.2312      0.          0.4624      0.39145563  0.39145563
   0.2312      0.2312      0.          0.29771209  0.29771209  0.2312    ]]


In [99]:
#算出個別文章的特徵值排序
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(y)  # titile 放文本
indices = np.argsort(X.toarray()[2])[::-1] #[::-1] hello倒著寫 → olleh  # X.toarray()[2] 是指第幾篇文章的意思
features = vectorizer.get_feature_names()
top_n = 3
top_features = [features[i] for i in indices[:top_n]]    # indices[:top_n]] 這邊改成單獨的 list sort 就可以抓出單篇的 TF-IDF 

print X.toarray()[0]

for i in top_features:
    print i

[ 0.          0.35653519  0.          0.35653519  0.          0.
  0.35653519  0.35653519  0.60366655  0.          0.          0.35653519]
上櫃
另外
九齊


In [ ]:
a = [2, 4, 3, 5, 1]
print sorted(a)

#a.sort()
print a
print "----------------------"
b=[[2, 4, 3, 5, 1],
   [3, 4, 3, 10, 1],
   [2, 4, 7, 5, 1],
   [2, 8, 3, 6, 1],  
]
print sorted(b)